In [1]:
# On affiche les graphiques dans le notebook en statique
%matplotlib inline

In [2]:
import numpy as np
import logging
import sys
import os
import gc
import joblib
import lightgbm as lgb
import optuna
import plotly
import kaleido
import mlflow

import pandas as pd
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import confusion_matrix, make_scorer

In [3]:
from src.p7_constantes import DATA_INTERIM, DATA_BASE, MODEL_DIR
from src.p7_constantes import LOCAL_HOST, LOCAL_PORT
from src.p7_util import timer, clean_ram
from src.p7_regex import sel_var

In [4]:
print("mlflow", mlflow.__version__)
print("optuna", optuna.__version__)
print("numpy", np.__version__)
print("plotly", plotly.__version__)
print("kaleido", kaleido.__version__)

mlflow 2.12.1
optuna 3.6.1
numpy 1.26.4
plotly 5.21.0
kaleido 0.1.0


Démarrer

In [5]:
# from src.p7_regex import sel_var

In [6]:
subdir = "light_simple/"

In [7]:
selected_features = joblib.load(os.path.join(MODEL_DIR, subdir, "selected_features_03.pkl"))
train = pd.read_csv(os.path.join(DATA_INTERIM, "train_simple_reduced_03.csv"), usecols=['SK_ID_CURR', 'TARGET'] + selected_features)
print("Forme de train.csv :", train.shape)
to_drop = sel_var(train.columns, 'Unnamed')
if to_drop:
    train = train.drop(to_drop, axis=1)
train.head()

Forme de train.csv : (191616, 19)
0 variables à inclure correspondant au motif 'Unnamed' : []
0 variables à exclure correspondant au motif 'None' : []
0 variables sélectionnées : []


,SK_ID_CURR,TARGET,AMT_ANNUITY,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_ID_PUBLISH,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,ANNUITY_INCOME_PERC,PAYMENT_RATE,BURO_AMT_CREDIT_MAX_OVERDUE_MEAN,ACTIVE_DAYS_CREDIT_MAX,ACTIVE_DAYS_CREDIT_ENDDATE_MIN,CLOSED_DAYS_CREDIT_MAX,CLOSED_DAYS_CREDIT_ENDDATE_MAX,PREV_CNT_PAYMENT_MEAN,APPROVED_CNT_PAYMENT_MEAN,INSTAL_DPD_MEAN
0,100002,1.0,24700.5,-9461,-637.0,-2120,0.083037,0.262949,0.139376,0.121978,0.060749,1681.029,-103.0,780.0,-476.0,85.0,24.0,24.0,0.000000
1,100003,0.0,35698.5,-16765,-1188.0,-291,0.311267,0.622246,NaN,0.132217,0.027598,0.000,-606.0,1216.0,-775.0,-420.0,10.0,10.0,0.000000
2,100008,0.0,27517.5,-16941,-1588.0,-477,NaN,0.354225,0.621226,0.277955,0.056101,0.000,-78.0,471.0,-1097.0,-792.0,14.0,14.0,37.628571
3,100009,0.0,41301.0,-13778,-3130.0,-619,0.774761,0.724000,0.492060,0.241526,0.026463,0.000,-239.0,-209.0,-684.0,1044.0,8.0,8.0,0.137255
4,100011,0.0,33826.5,-20099,NaN,-3514,0.587334,0.205747,0.751724,0.300680,0.033176,5073.615,NaN,NaN,-1309.0,-860.0,14.0,14.0,7.946154


In [8]:
#from src.p7_util import clean_ram

In [9]:
dic_local = locals()
to_del = ['to_drop', 'selected_features']
clean_ram(to_del, dic_local)

2 variables détruites : ['to_drop', 'selected_features']


In [10]:
# Passe les variables catégorielles en dtype category sinon LGM ne pourra pas les traiter
cat_features = list(train.loc[:, train.dtypes == 'object'].columns.values)
for feature in cat_features:
    train[feature] = pd.Series(train[feature], dtype="category")

In [11]:
# Essai sur un échantillon d'abord
n_rows = 30_000
X = train.drop(columns=["SK_ID_CURR", "TARGET"], axis=1)
y = train["TARGET"]
if n_rows:
    X = X.head(n_rows)
    y = y.head(n_rows)

In [12]:
predictors = list(X.columns)
predictors

['AMT_ANNUITY',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_ID_PUBLISH',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'ANNUITY_INCOME_PERC',
 'PAYMENT_RATE',
 'BURO_AMT_CREDIT_MAX_OVERDUE_MEAN',
 'ACTIVE_DAYS_CREDIT_MAX',
 'ACTIVE_DAYS_CREDIT_ENDDATE_MIN',
 'CLOSED_DAYS_CREDIT_MAX',
 'CLOSED_DAYS_CREDIT_ENDDATE_MAX',
 'PREV_CNT_PAYMENT_MEAN',
 'APPROVED_CNT_PAYMENT_MEAN',
 'INSTAL_DPD_MEAN']

In [13]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 17 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   AMT_ANNUITY                       30000 non-null  float64
 1   DAYS_BIRTH                        30000 non-null  int64  
 2   DAYS_EMPLOYED                     27690 non-null  float64
 3   DAYS_ID_PUBLISH                   30000 non-null  int64  
 4   EXT_SOURCE_1                      20974 non-null  float64
 5   EXT_SOURCE_2                      29959 non-null  float64
 6   EXT_SOURCE_3                      25657 non-null  float64
 7   ANNUITY_INCOME_PERC               30000 non-null  float64
 8   PAYMENT_RATE                      30000 non-null  float64
 9   BURO_AMT_CREDIT_MAX_OVERDUE_MEAN  21128 non-null  float64
 10  ACTIVE_DAYS_CREDIT_MAX            23736 non-null  float64
 11  ACTIVE_DAYS_CREDIT_ENDDATE_MIN    23015 non-null  float64
 12  CLOS

# Métrique personnalisée

In [14]:

# Pénalise les Faux Negatifs dans le F1 Score
def weight_f1(y_true, y_pred, weight_fn=10):
    _, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    # fn = Nombre de faux négatifs (oubli de prédire un défaut)
    # fp = Nombre de faux positifs (défaut prédit à tort)
    
    # f1 standard = 2 * tp / (2 * tp + fp + fn)
    
    #weighted_f1 = 2 * tp / (2 * tp + weight_fn * fn  + (1 - weight_fn) * fp)
    weighted_f1 = 2 * tp / (2 * tp + (weight_fn * fn  +  fp) / weight_fn)
    
    return weighted_f1

# Créer une métrique personnalisée à partir de la fonction de perte
custom_f1 = make_scorer(weight_f1, greater_is_better=True)

## Run A Study

In [15]:
"""
La fonction _objectif est appelée une fois pour chaque essai (trial).
Ici on entraîne un LGBMClassfier et on calcule la métrique : ? replacer le f1-score par une autre
Optuna passe un objet trial à la fonction _objectif, qu'on peut utiliser pour en définir les paramètres.
log=True : applique une log scale aux valeurs à tester dans l'étendue spécifiée (pour les valeurs num), 
Effet : plus de valeurs sont testées à proximité de la borne basse et moins (logarithmiquement) vers
la borne haute
Convient particulièrement bien au learning rate : on veut se concentrer sur des valeurs + petites et 
augmenter exponentiellement le pas des valeurs à tester pour les plus grandes

Le pruning callbac est le mécanisme qui applique le pruning dynamique pendant l'entraînement du modèle LightGBM 
dans le cadre d'une étude Optuna. 
1 - Initialisation du callback : 
Crée le callback de pruning qui surveillera l'entraînement du modèle LightGBM 
et effectuera le pruning selon les instructions spécifiées par l'étude Optuna
2 - Évaluation périodique : 
Pendant l'entraînement de LightGBM, le callback est appelé périodiquement
pour évaluer les performances du modèle en fonction des critères de pruning définis par Optuna.
3 - Décision de pruning : 
Le callback utilise les informations fournies par Optuna, 
telles que les valeurs d'objectif de l'essai actuel et les valeurs d'objectif des essais précédents, 
pour décider si l'essai actuel doit être pruned en fonction de son efficacité par rapport à d'autres essais.
4 - Pruning : 
Pruning appliqué si décidé (arrête l'entraînement de ce modèle LightGBM)
5 - Réévaluation et ajustement :
Après le pruning d'un essai, l'entraînement peut continuer avec les essais restants, 
et le processus de pruning peut être répété à intervalles réguliers jusqu'à ce que l'étude soit terminée.


"""

"\nLa fonction _objectif est appelée une fois pour chaque essai (trial).\nIci on entraîne un LGBMClassfier et on calcule la métrique : ? replacer le f1-score par une autre\nOptuna passe un objet trial à la fonction _objectif, qu'on peut utiliser pour en définir les paramètres.\nlog=True : applique une log scale aux valeurs à tester dans l'étendue spécifiée (pour les valeurs num), \nEffet : plus de valeurs sont testées à proximité de la borne basse et moins (logarithmiquement) vers\nla borne haute\nConvient particulièrement bien au learning rate : on veut se concentrer sur des valeurs + petites et \naugmenter exponentiellement le pas des valeurs à tester pour les plus grandes\n\nLe pruning callbac est le mécanisme qui applique le pruning dynamique pendant l'entraînement du modèle LightGBM \ndans le cadre d'une étude Optuna. \n1 - Initialisation du callback : \nCrée le callback de pruning qui surveillera l'entraînement du modèle LightGBM \net effectuera le pruning selon les instructions 

In [16]:
#from sklearn.model_selection import cross_validate

In [17]:
# Nécessite l'installation optuna-integration
def objective(optimize_boosting_type=True):
    def _objective(trial):
        if optimize_boosting_type:
            boosting_type = trial.suggest_categorical("boosting_type", ["dart", "gbdt"])
        else:
            boosting_type = "gbdt"
        lambda_l1 = trial.suggest_float(
            'lambda_l1', 1e-8, 10.0, log=True),
        lambda_l2 = trial.suggest_float(
            'lambda_l2', 1e-8, 10.0, log=True),
        num_leaves = trial.suggest_int(
            'num_leaves', 2, 256),
        feature_fraction = trial.suggest_float(
            'feature_fraction', 0.4, 1.0),
        bagging_fraction = trial.suggest_float(
            'bagging_fraction', 0.4, 1.0),
        bagging_freq = trial.suggest_int(
            'bagging_freq', 1, 7),
        min_child_samples = trial.suggest_int(
            'min_child_samples', 5, 100),
        learning_rate = trial.suggest_float(
            "learning_rate", 0.0001, 0.5, log=True),
        max_bin = trial.suggest_int(
            "max_bin", 128, 512, step=32)
        n_estimators = trial.suggest_int(
            "n_estimators", 40, 400, step=20)
        
        # 'binary' est la métrique d'erreur
        pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "binary")
        
        model = lgb.LGBMClassifier(
            force_row_wise=True,
            boosting_type=boosting_type,
            n_estimators=n_estimators,
            lambda_l1=lambda_l1,
            lambda_l2=lambda_l2,
            num_leaves=num_leaves,
            feature_fraction=feature_fraction,
            bagging_fraction=bagging_fraction,
            bagging_freq=bagging_freq,
            min_child_samples=min_child_samples,
            learning_rate=learning_rate,
            max_bin=max_bin,
            callbacks=[pruning_callback],
            verbose=-1,
            )
        
        
        # Si on voulait traquer avec mlflow différents scores :
        # Pour intégration optuna mlflow avec nested runs voir :
        # https://mlflow.org/docs/latest/traditional-ml/hyperparameter-tuning-with-child-runs/notebooks/hyperparameter-tuning-with-child-runs.html?highlight=run%20description
        """scoring = {
            "f1_macro": "f1_macro",
            "recall_weighted": "recall_weighted",
            "roc_auc": "roc_auc",
            "balanced_accuracy": "balanced_accuracy",
            }
        scores = cross_validate(model, X, y, scoring=scoring, cv=5)
        mean_f1_macro = scores["f1_macro"].mean()
        mean_roc_auc = scores["test_roc_auc"].mean()
        mean_balanced_accuracy = scores["test_balanced_accuracy"].mean()
        mean_recall_weighted = scores["test_recall_weighted"].mean()
        return mean_recall_weighted.mean()"""

        # Ici on ne choisit qu'une métrique
        scores = cross_val_score(model, X, y, scoring="f1_macro", cv=5)
        return scores.mean()

    return _objective

In [18]:
"""
Métriques pour notre cas :

Le Recall mesure la proportion de vrais positifs (défauts correctement prédits)
parmi tous les vrais positifs et faux négatifs (défauts réels).
En choisissant le Recall comme métrique, on se concentre sur la capacité du modèle à capturer la majorité des cas de défauts, 
minimisant ainsi le risque de faux négatifs (ne pas prédire un défaut lorsqu'il existe réellement).

Le F1-score est une mesure combinée de la précision et du recall. 
Il s'agit de la moyenne harmonique de la précision et du recall.
Le F1-score est particulièrement utile lorsque les classes sont déséquilibrées car il donne plus de poids aux classes minoritaires. 
Ainsi, il peut être une bonne métrique lorsqu'on veut trouver un compromis entre la précision et le recall.

AUC-ROC (Area Under the Receiver Operating Characteristic Curve) : L'AUC-ROC est une métrique qui mesure la capacité du modèle à classer
correctement les exemples positifs et négatifs. 
Elle est robuste aux déséquilibres de classe et donne une indication de la capacité du modèle à discriminer entre les classes.
"""

"\nMétriques pour notre cas :\n\nLe Recall mesure la proportion de vrais positifs (défauts correctement prédits)\nparmi tous les vrais positifs et faux négatifs (défauts réels).\nEn choisissant le Recall comme métrique, on se concentre sur la capacité du modèle à capturer la majorité des cas de défauts, \nminimisant ainsi le risque de faux négatifs (ne pas prédire un défaut lorsqu'il existe réellement).\n\nLe F1-score est une mesure combinée de la précision et du recall. \nIl s'agit de la moyenne harmonique de la précision et du recall.\nLe F1-score est particulièrement utile lorsque les classes sont déséquilibrées car il donne plus de poids aux classes minoritaires. \nAinsi, il peut être une bonne métrique lorsqu'on veut trouver un compromis entre la précision et le recall.\n\nAUC-ROC (Area Under the Receiver Operating Characteristic Curve) : L'AUC-ROC est une métrique qui mesure la capacité du modèle à classer\ncorrectement les exemples positifs et négatifs. \nElle est robuste aux dé

In [19]:
"""
scoring avec cross_val_score :
The 'scoring' parameter of cross_val_score must be a str among pour la classification :
{
'accuracy', 'balanced_accuracy', 'f1', 
'jaccard_micro',
'jaccard_weighted',
'precision_micro',
'recall_samples', 
'precision_macro',
'jaccard', 
'recall_micro', 
'roc_auc_ovo', 
'f1_samples', 'recall_macro', 'precision_weighted', 'f1_weighted', 'roc_auc_ovr_weighted',
'f1_micro', 'roc_auc', 'f1_macro', 'precision_samples', 'recall', 'jaccard_samples', 'precision', 'average_precision', 'top_k_accuracy', 
'neg_log_loss', 'jaccard_macro', 'neg_brier_score', 'roc_auc_ovr', 'recall_weighted', 'roc_auc_ovo_weighted'
}, a callable or None.

"""

"\nscoring avec cross_val_score :\nThe 'scoring' parameter of cross_val_score must be a str among pour la classification :\n{\n'accuracy', 'balanced_accuracy', 'f1', \n'jaccard_micro',\n'jaccard_weighted',\n'precision_micro',\n'recall_samples', \n'precision_macro',\n'jaccard', \n'recall_micro', \n'roc_auc_ovo', \n'f1_samples', 'recall_macro', 'precision_weighted', 'f1_weighted', 'roc_auc_ovr_weighted',\n'f1_micro', 'roc_auc', 'f1_macro', 'precision_samples', 'recall', 'jaccard_samples', 'precision', 'average_precision', 'top_k_accuracy', \n'neg_log_loss', 'jaccard_macro', 'neg_brier_score', 'roc_auc_ovr', 'recall_weighted', 'roc_auc_ovo_weighted'\n}, a callable or None.\n\n"

In [20]:
"""
balanced accuracy
The balanced accuracy in binary and multiclass classification problems to deal with imbalanced datasets. 
It is defined as the average of recall obtained on each class.
The best value is 1 and the worst value is 0 when adjusted=False.
"""

'\nbalanced accuracy\nThe balanced accuracy in binary and multiclass classification problems to deal with imbalanced datasets. \nIt is defined as the average of recall obtained on each class.\nThe best value is 1 and the worst value is 0 when adjusted=False.\n'

In [21]:
cat_features

[]

In [22]:
"""
Pruning (="Elagage") = technique pour arrêter prématurément l'exécution (train + eval) si 
il est peu probable que l'essai conduise à une amélioreration significative des performances
(l'entraînement est stoppé si l'algo sous-performe)

**** HYPERBAND

Hyperband est une technique de pruning qui combine le pruning de type "Successive halving" et
d'autres techniques (random search, multi-bracket resource allocation strategy).

1 - Successive Halving (Demi-seuccessifs):
Au début de l'optimisation, Hyperband crée un grand nombre de configurations d'hyperparamètres
et entraîne chaque modèle pendant un petit nombre d'itérations (eochs). 
Ensuite, il évalue les performances de ces modèles et conserve uniquement les meilleurs. 
Il répète ce processus plusieurs fois, en doublant à chaque fois le nombre d'itérations, 
mais en réduisant de moitié le nombre de configurations conservées.

Ainsi les modèles prometteurs bénéficient de plus d'itérations pour converger vers de bonnes performances,
tandis que les modèles moins prometteurs sont élagués à chaque étape.

2 - Pruning basé sur la performance:
À chaque étape du Successive Halving, Hyperband évalue périodiquement les performances des modèles en formation
et décide de "pruner" les modèles moins prometteurs. 
Les modèles qui n'atteignent pas un certain seuil de performance sont arrêtés prématurément.

*** MULTI_BRACKET RESSOURCE ALLOCATION strategy (Allocation de ressource à plusieurs 'brackets' ou niveaux / échelon)

divise les ressources disponibles (telles que le nombre d'itérations, d'évaluations de modèles, etc.)
en plusieurs "brackets" ou "niveaux" et alloue ces ressources de manière dynamique en fonction de la performance
des modèles à chaque niveau.

1 - Multi-brackets
L'algorithme divise le budget total de ressources (par exemple, le nombre maximal d'itérations ou le temps total de calcul) en plusieurs "brackets" ou "échelons".
Chaque bracket représente une étape distincte de l'algorithme où différents ensembles d'hyperparamètres sont évalués.
Chaque bracket est caractérisé par un budget de ressources spécifique, qui peut être soit le nombre d'itérations, soit le temps de calcul. 
Les brackets initiaux ont un budget élevé tandis que les brackets ultérieurs ont des budgets plus bas.

2 - Resource allocation
Alloue dynamiquement les ressources aux config d'hyperparams
Dans chaque bracket, Hyperband commence par évaluer un grand nombre de configurations d'hyperparamètres pour un nombre restreint d'itérations. 
Ensuite, il élimine les configurations sous-performantes et alloue davantage de ressources aux configurations les plus prometteuses.

Within each bracket, successive halving is applied to iteratively eliminate underperforming configurations and
allocate more resources to the remaining promising ones.
At the begining of each bracket, a new set of hyperparam configurations is sampled using random search.

=> reduce risk of missing good config
more efficient and effective hyperparam tuning.
"""

'\nPruning (="Elagage") = technique pour arrêter prématurément l\'exécution (train + eval) si \nil est peu probable que l\'essai conduise à une amélioreration significative des performances\n(l\'entraînement est stoppé si l\'algo sous-performe)\n\n**** HYPERBAND\n\nHyperband est une technique de pruning qui combine le pruning de type "Successive halving" et\nd\'autres techniques (random search, multi-bracket resource allocation strategy).\n\n1 - Successive Halving (Demi-seuccessifs):\nAu début de l\'optimisation, Hyperband crée un grand nombre de configurations d\'hyperparamètres\net entraîne chaque modèle pendant un petit nombre d\'itérations (eochs). \nEnsuite, il évalue les performances de ces modèles et conserve uniquement les meilleurs. \nIl répète ce processus plusieurs fois, en doublant à chaque fois le nombre d\'itérations, \nmais en réduisant de moitié le nombre de configurations conservées.\n\nAinsi les modèles prometteurs bénéficient de plus d\'itérations pour converger vers

In [23]:
"""
SAMPLER TPE :
Tree-structured Parzen Estimator
TPESampler utilise une approche bayésienne pour estimer les distributions de probabilité des valeurs d'hyperparamètres,
en se basant sur les performances des essais précédents, 
afin de guider efficacement la recherche vers des régions prometteuses de l'espace des hyperparamètres.

1 - Initialisation des distributions :
L'algorithme TPE commence par définir des distributions de probabilité pour chaque hyperparamètre à optimiser.
Ces distributions peuvent être continues (comme des distributions gaussiennes) ou discrètes (comme des distributions uniformes).
Pour chaque hyperparamètre, deux distributions sont définies : 
une pour les valeurs considérées comme "bonnes" (positive), et une pour les valeurs considérées comme "mauvaises" (négative).

2 - Évaluation des essais :
À chaque étape de l'optimisation, 
TPE utilise les essais précédents pour estimer les distributions de probabilité des valeurs d'hyperparamètres.
Les essais sont divisés en deux groupes : 
ceux qui ont produit de bonnes performances et ceux qui ont produit de mauvaises performances, 
en fonction du critère d'objectif (par exemple, la précision d'un modèle).
Les distributions de probabilité sont mises à jour en utilisant les essais des deux groupes,
en ajustant les paramètres des distributions pour mieux modéliser les valeurs d'hyperparamètres qui ont conduit à de bonnes performances.

3 - Échantillonnage des essais :
Une fois que les distributions de probabilité ont été mises à jour, 
TPE échantillonne de nouvelles valeurs d'hyperparamètres à partir de ces distributions. 
Les valeurs échantillonnées sont généralement celles qui maximisent l'espérance d'une fonction d'acquisition 
(par exemple, l'espérance de l'amélioration de l'objectif) 
ou qui minimisent une fonction de coût (par exemple, l'espérance de la perte de l'objectif).

4 - Évaluation des performances :
Les nouvelles valeurs d'hyperparamètres échantillonnées sont utilisées pour effectuer de nouveaux essais.
Les performances de ces essais sont évaluées à l'aide de la fonction d'objectif, 
et les résultats sont utilisés pour mettre à jour les distributions de probabilité et répéter le processus.
"""

'\nSAMPLER TPE :\nTree-structured Parzen Estimator\nTPESampler utilise une approche bayésienne pour estimer les distributions de probabilité des valeurs d\'hyperparamètres,\nen se basant sur les performances des essais précédents, \nafin de guider efficacement la recherche vers des régions prometteuses de l\'espace des hyperparamètres.\n\n1 - Initialisation des distributions :\nL\'algorithme TPE commence par définir des distributions de probabilité pour chaque hyperparamètre à optimiser.\nCes distributions peuvent être continues (comme des distributions gaussiennes) ou discrètes (comme des distributions uniformes).\nPour chaque hyperparamètre, deux distributions sont définies : \nune pour les valeurs considérées comme "bonnes" (positive), et une pour les valeurs considérées comme "mauvaises" (négative).\n\n2 - Évaluation des essais :\nÀ chaque étape de l\'optimisation, \nTPE utilise les essais précédents pour estimer les distributions de probabilité des valeurs d\'hyperparamètres.\nLes

In [24]:
"""
Démarrer un serveur mlflow local en ligne de commande :
mlflow server --host 127.0.0.1 --port 8080
"""

'\nDémarrer un serveur mlflow local en ligne de commande :\nmlflow server --host 127.0.0.1 --port 8080\n'

In [25]:
# Use the fluent API to set the tracking uri and the active experiment
print(f"{LOCAL_HOST}:{LOCAL_PORT}")

http://127.0.0.1:8080


In [25]:
# Use the fluent API to set the tracking uri and the active experiment
mlflow.set_tracking_uri(f"{LOCAL_HOST}:{LOCAL_PORT}")

In [27]:
# On crèe une epérience
experiment_name = "lightgbm_hyperparam"

experiment_description = (
    "Recherche d'hyperparamètres pour le modèle LightGBM"
    "Recherche Bayesienne - mono objectif - Hyperband"
)

experiment_tags = {
    "model": "lightgbm",
    "task": "hyperparam",
    "mlflow.note.content": experiment_description,
}

experiment_id = mlflow.create_experiment(
    experiment_name,
    #artifact_location=os.path.join(MODEL_DIR, subdir).as_uri(),
    artifact_location=os.path.join(MODEL_DIR, subdir),
    tags=experiment_tags,
)

In [28]:
# On active l'expérience
experiment_metadata = mlflow.set_experiment(experiment_id=experiment_id)
experiment_metadata

<Experiment: artifact_location='file:///E:/Mes Documents/_Open Classroom/Code/p7/models/light_simple', creation_time=1714818682024, experiment_id='960899003815379867', last_update_time=1714818682024, lifecycle_stage='active', name='lightgbm_hyperparam_test', tags={'mlflow.note.content': "Recherche d'hyperparamètres pour le modèle "
                        'LightGBMRecherche Bayesienne - mono objectif - '
                        'Hyperband',
 'model': 'lightgbm',
 'task': 'hyperparam'}>

In [53]:
print(experiment_metadata.name)
print(experiment_metadata.lifecycle_stage)

lightgbm_hyperparam_test
active


In [29]:
with timer("Optimize hyperparameters"):
    # Utilise l'algorithme d'optimisation TPE (Tree-structured Parzen Estimator) comme méthode d'échantillonnage
    # Il s'agit de l'algo qui génère les valeurs des hyperparams lors de chaque essai d'optimisation
    # Ici il est utilisé en conjonction avec le pruning Hyperband
    # => Le sampler choisit les params à essayer, le pruner les arrête prématurément si non performants
    sampler = optuna.samplers.TPESampler()
    
    # Optuna a réalisé plusieurs études empiriques avec différents algorithmes de pruning.
    # Empiriquement, l'algorithme Hyperband a donné les meilleurs résultats
    # Voir : https://github.com/optuna/optuna/wiki/Benchmarks-with-Kurobako
    # reduction_factor contrôle combien de trials sont proposés dans chaque Halving Round
    pruner = optuna.pruners.HyperbandPruner(
        min_resource=10, max_resource=400, reduction_factor=3)
    
    # On crèe une run MLFlow
    run_name ="lightgbm_single_f1_all"
    run_description ='Single objective - métrique f1_macro - all data kernel simple'
    
    
    with mlflow.start_run(run_name=run_name) as run:
        # description du run
        mlflow.set_tag("mlflow.note.content", run_description)
        
        study = optuna.create_study(
            direction='maximize', 
            sampler=sampler,
            pruner=pruner,
            study_name="lightgbm_single_obj_03",
            #storage=os.path.join(MODEL_DIR, subdir)
        )
    
        # gc appelle le garbage collector après chaque trial
        study.optimize(objective(), n_trials=10, gc_after_trial=True, n_jobs=-1)
        
        best_params = study.best_trial.params
        mlflow.log_params(best_params)
        #mlflow.sklearn.log_model(study.model, "Meilleur modèle")
        
        fig = optuna.visualization.plot_parallel_coordinate(study, params=["boosting_type", "num_leaves", "learning_rate", "n_estimators"])
        im_dir = os.path.join(MODEL_DIR, subdir)
        #fig.write_image(file=os.path.join(im_dir, "single_parallel_coordinates.png"), format="png", scale=6)
        fig.write_html(os.path.join(im_dir, "single_parallel_coordinates.html"))
        fig = optuna.visualization.plot_param_importances(study)
        #fig.write_image(file=os.path.join(im_dir, "single_hyperparam_importance.png"), format="png", scale=1)
        fig.write_html(os.path.join(im_dir, "single_hyperparam_importance.html"))
        
        mlflow.log_artifact(os.path.join(im_dir, "single_parallel_coordinates.html"))
        mlflow.log_artifact(os.path.join(im_dir, "single_hyperparam_importance.html"))
        #mlflow.log_artifact(os.path.join(im_dir, "single_hyperparam_importance.png"))
    # Force mlflow à terminer le run même s'il y a une erreur dedans
    mlflow.end_run()

[I 2024-05-04 12:31:50,233] A new study created in memory with name: lightgbm_single_obj_03
[I 2024-05-04 12:31:52,452] Trial 5 finished with value: 0.0 and parameters: {'boosting_type': 'dart', 'lambda_l1': 6.823135770672805e-06, 'lambda_l2': 0.007499118161723124, 'num_leaves': 30, 'feature_fraction': 0.8147347003671239, 'bagging_fraction': 0.6515787288641275, 'bagging_freq': 7, 'min_child_samples': 51, 'learning_rate': 0.010886880720281183, 'max_bin': 256, 'n_estimators': 40}. Best is trial 5 with value: 0.0.
[I 2024-05-04 12:31:54,235] Trial 0 finished with value: 0.5987567489425044 and parameters: {'boosting_type': 'gbdt', 'lambda_l1': 0.03762610517308741, 'lambda_l2': 5.844813698530148e-07, 'num_leaves': 21, 'feature_fraction': 0.7122226646460588, 'bagging_fraction': 0.8924020558960004, 'bagging_freq': 1, 'min_child_samples': 69, 'learning_rate': 0.25307716027209676, 'max_bin': 480, 'n_estimators': 120}. Best is trial 0 with value: 0.5987567489425044.
[I 2024-05-04 12:31:58,211] T

Optimize hyperparameters - duration (hh:mm:ss) : 0:00:35


In [80]:
mlflow.end_run()

In [50]:
print(study.best_trial)
best_params = study.best_trial.params
for k, v in best_params.items():
    print(f"{k} : {v}")

FrozenTrial(number=7, state=TrialState.COMPLETE, values=[0.6713640280500157], datetime_start=datetime.datetime(2024, 5, 4, 11, 22, 20, 556119), datetime_complete=datetime.datetime(2024, 5, 4, 11, 22, 31, 369550), params={'boosting_type': 'dart', 'lambda_l1': 1.3159775456532645, 'lambda_l2': 7.427708103108906e-05, 'num_leaves': 95, 'feature_fraction': 0.9301882478779776, 'bagging_fraction': 0.6469799790785027, 'bagging_freq': 1, 'min_child_samples': 90, 'learning_rate': 0.17514302089624703, 'max_bin': 288, 'n_estimators': 120}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'boosting_type': CategoricalDistribution(choices=('dart', 'gbdt')), 'lambda_l1': FloatDistribution(high=10.0, log=True, low=1e-08, step=None), 'lambda_l2': FloatDistribution(high=10.0, log=True, low=1e-08, step=None), 'num_leaves': IntDistribution(high=256, log=False, low=2, step=1), 'feature_fraction': FloatDistribution(high=1.0, log=False, low=0.4, step=None), 'bagging_fraction': FloatDistri

In [70]:
joblib.dump(study, os.path.join(MODEL_DIR, subdir, "opt_lightgbm_single_03.pkl"))

['models/light_simple/opt_lightgbm_single_03.pkl']

## Understanding Parameters

In [72]:
study = joblib.load(os.path.join(MODEL_DIR, subdir, "opt_lightgbm_single_03.pkl"))

In [73]:
# Nécessite l'installation de plotly et de Kaleido (redémarrer le kernel impérativement après plotly, kaleido version 0.1.0 et nformat version récente)
# pip install kaleido==0.1.0
# pip install --upgrade nbformat
# Ajouter le path de kaleido.cmd au PATH windows
# Tout redémarrer
fig = optuna.visualization.plot_parallel_coordinate(study, params=["boosting_type", "num_leaves", "learning_rate", "n_estimators"])
joblib.dump(fig, os.path.join(MODEL_DIR,'fig_plotly.pkl'))
print(type(fig))

<class 'plotly.graph_objs._figure.Figure'>


In [74]:
im_dir = os.path.join(MODEL_DIR, subdir)

In [75]:
fig.write_image(file=os.path.join(im_dir, "single_parallel_coordinates.png"), format="png", scale=6)

In [76]:
fig.show()

In [44]:
# Plus rapide en html
im_path = os.path.join(im_dir, "single_parallel_coordinates.html")
fig.write_html(im_path)
fig.show()

In [78]:
fig = optuna.visualization.plot_param_importances(study)
fig.write_image(file=os.path.join(im_dir, "single_hyperparam_importance.png"), format="png", scale=6)
fig.show()

## Multi-objective optimization

In [22]:
def moo_objective(trial):
    learning_rate = trial.suggest_float("learning_rate", 0.0001, 0.5, log=True),

    model = lgb.LGBMClassifier(
        force_row_wise=True,
        boosting_type='gbdt',
        n_estimators=200,
        lambda_l1=3.298803078077973e-07,
        lambda_l2=8.938532783741386e-07,
        num_leaves=6,
        feature_fraction=0.5133218336120866,
        bagging_fraction=0.9660809666082303,
        bagging_freq=7,
        min_child_samples=91,
        learning_rate=learning_rate,
        max_bin=320,
        verbose=-1,
    )
    scores = cross_val_score(model, X, y, scoring="f1_macro")
    return learning_rate[0], scores.mean()

In [23]:
study = optuna.create_study(directions=["maximize", "maximize"])
study.optimize(moo_objective, n_trials=100)

[I 2023-04-27 10:47:35,501] A new study created in memory with name: no-name-eb122531-f2ee-4cb5-92d1-8efba6a28eef
[I 2023-04-27 10:47:36,221] Trial 0 finished with values: [0.018276625572235056, 0.7229728382357858] and parameters: {'learning_rate': 0.018276625572235056}. 
[I 2023-04-27 10:47:36,922] Trial 1 finished with values: [0.012281350875017864, 0.713129498994182] and parameters: {'learning_rate': 0.012281350875017864}. 
[I 2023-04-27 10:47:37,601] Trial 2 finished with values: [0.3179931543219117, 0.7181352358216049] and parameters: {'learning_rate': 0.3179931543219117}. 
[I 2023-04-27 10:47:38,317] Trial 3 finished with values: [0.3317048028124761, 0.7182462105095208] and parameters: {'learning_rate': 0.3317048028124761}. 
[I 2023-04-27 10:47:39,042] Trial 4 finished with values: [0.010097499163932589, 0.689595604293249] and parameters: {'learning_rate': 0.010097499163932589}. 
[I 2023-04-27 10:47:39,716] Trial 5 finished with values: [0.0010284892903758126, 0.4235082247366586]

In [24]:
fig = optuna.visualization.plot_pareto_front(study, target_names=["learning_rate", "f1"])
fig.write_image(file="figures/ch5_pareto.png", format="png", scale=6)
fig.show()